## Installing BeautifulSoup and other necessary packages 

In [1]:
!pip install beautifulsoup4

!pip install lxml

!pip install html5lib

!pip install requests

## Importing initial packages

In [2]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import numpy as np
import csv 

## Scraping raw data from the Wiki source and transforming it using the soup

In [3]:
raw_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(raw_data, 'lxml')

## Extracting data from the Postal Codes table and appending it into lists 

In [4]:
table = soup.find('table', class_='wikitable') # Identifies table containing the postal codes
rows = table.find_all('tr') # Extracts all rows from the table 

# Initializes lists 
postalcodes = []
boroughs = []
neighborhoods = []

for row in rows:    
    columns = row.find_all('td')
    try:
        if columns[1].text.split('\n')[0] != 'Not assigned': # ignores boroughs named "Not Assignmend"
            
            postalcode = columns[0].text.split('\n')[0] # ignores newline character at the end
            postalcodes.append(postalcode)
             
        
            borough = columns[1].text.split('\n')[0]  # ignores newline character at the end
            boroughs.append(borough)
        
            neighborhood = columns[2].text.replace('/',',').split('\n')[0] # replaces / with , for visual reasons
            neighborhoods.append(neighborhood)
            
    except Exception as e:
        pass 
        
        

## Join the above 3 lists into a 'df' Dataframe

In [5]:
df = pd.DataFrame(list(zip(postalcodes, boroughs,neighborhoods)), columns =['PostalCode', 'Borough','Neighborhood']) 
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


## Checks for the shape

In [6]:
df.shape

(103, 3)

## Extracting the 'PostalCode' column from df 

In [8]:
df_postals = df['PostalCode'].values
df_postals

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

## Geocoder API was very unreliable in this exercise, therefore the free OpenCage Geocoder was used 

### OpenCage API Credentials

In [9]:
API_KEY = '5e232949c48244eb888c484b236c9534'

In [11]:
import json

latitudes = [] #initializes empty list to append latitude data
longitudes = [] #initializes empty list to append longitude data

for postal in df_postals:
    location_input = postal + " Toronto, Canada" # correct location input format 
    url = 'https://api.opencagedata.com/geocode/v1/json?q={}&key={}'.format(location_input, API_KEY) # correct url format
    obj = json.loads(requests.get(url).text)  # Gets the json file, stores in a dictionary
    
    results = obj['results'] # gets the 'results' key
    latitude = results[0]['geometry']['lat'] # gets the 'latitude' data 
    longitude = results[0]['geometry']['lng'] #gets the 'longitude' data
    
    latitudes.append(latitude) # Appends data to the list of latitudes
    longitudes.append(longitude) # Appends data to the list of longitudes


## Appends these lists into the 'df' DataFrame

In [12]:
df['Latitude'] = latitudes 
df['Longitude'] = longitudes
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.653482,-79.383935
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.655500,-79.362600
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.722300,-79.450400
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.653482,-79.383935


## A free OpenCage account is limited to 2500 API calls/day. Therefore, need to save the results into a csv file as backup

In [13]:
df.to_csv('Toronto_Neighborhood.csv')

## Creates a pandas Dataframe from the backup csv file

In [14]:
df = pd.read_csv('Toronto_Neighborhood.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.653482,-79.383935
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.655500,-79.362600
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.722300,-79.450400
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.653482,-79.383935


## List number of Boroughs and Neighborhoods

In [15]:
print('The dataframe has {} unique Postal Codes, {} boroughs and {} neighborhoods.'.format(len(df['PostalCode'].unique()), 
        len(df['Borough'].unique()),
        len(df['Neighborhood'].unique())
    )
)

The dataframe has 103 unique Postal Codes, 10 boroughs and 98 neighborhoods.


## Installing and importing Geopy and Folium

In [16]:
!pip install geopy
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium
import folium

## Visualizing a map of Toronto with all Neighborhoods and Boroughs

In [17]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
to_latitude = location.latitude
to_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(to_latitude, to_longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[to_latitude, to_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Exploring Boroughs that contain the word "Etobicoke"

In [19]:
etobicoke_data = df[df['Borough'].str.contains("Etobicoke")].reset_index(drop=True) # searches for entries containing 'Etobicoke' only and resets the index
etobicoke_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.666200,-79.528200
1,M9B,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr...",43.640741,-79.541902
2,M9C,Etobicoke,"Eringate , Bloordale Gardens , Old Burnhamthor...",43.644110,-79.588907
3,M9P,Etobicoke,Westmount,43.694900,-79.532300
4,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.695166,-79.550890
5,M8V,Etobicoke,"New Toronto , Mimico South , Humber Bay Shores",43.607500,-79.501300
6,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.743200,-79.587600
7,M8W,Etobicoke,"Alderwood , Long Branch",43.602100,-79.540200
8,M9W,Etobicoke,Northwest,43.715486,-79.564072
9,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.651800,-79.507600


In [20]:
# create map of Etobicoke using latitude and longitude values
map_etobicoke = folium.Map(location=[to_latitude, to_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Borough'], etobicoke_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_etobicoke)  
    
map_etobicoke

## Foursquare API Credentials

In [21]:
CLIENT_ID = 'NVNSTQUGEPDCCEKZ3FKLOWND1FYPVIR3YDLAPQJPV5CYDWWK' # your Foursquare ID
CLIENT_SECRET = 'XG233VXSPMLJOQS40H23GWCGGWKNBGJWT1NLSNLDBZYPB4PQ' # your Foursquare Secret
VERSION = '20200401' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NVNSTQUGEPDCCEKZ3FKLOWND1FYPVIR3YDLAPQJPV5CYDWWK
CLIENT_SECRET:XG233VXSPMLJOQS40H23GWCGGWKNBGJWT1NLSNLDBZYPB4PQ


## Explores the first neighborhood in Etobicoke

In [22]:
etobicoke_data.loc[0, 'Neighborhood']

'Islington Avenue'

In [23]:
neighborhood_latitude = etobicoke_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = etobicoke_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Islington Avenue are 43.6662, -79.5282.


In [24]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NVNSTQUGEPDCCEKZ3FKLOWND1FYPVIR3YDLAPQJPV5CYDWWK&client_secret=XG233VXSPMLJOQS40H23GWCGGWKNBGJWT1NLSNLDBZYPB4PQ&v=20200401&ll=43.6662,-79.5282&radius=500&limit=100'

## Makes the 'get' request and acquires the results

In [25]:
results = requests.get(url).json()

## Defines function to acquire the categories and venues of the reults 

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Cleans and prepares the json object and transforms it into a pandas dataframe

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,TD Canada Trust,Bank,43.662545,-79.531749
1,Shoppers Drug Mart,Pharmacy,43.663067,-79.531753
2,Humber Valley Park,Park,43.664825,-79.524999
3,Humber Valley Rink,Skating Rink,43.664826,-79.524873
4,Thorncrest Drug Store,Pharmacy,43.662988,-79.531817


## Prints the number of unique venues found by FourSquare

In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


## Creates functions to repeat this process for other neighborhoods in Etobicoke

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Uses the code on each neighborhood to create a new pandas dataframe 

In [30]:
etobicoke_venues = getNearbyVenues(names=etobicoke_data['Neighborhood'],
                                   latitudes=etobicoke_data['Latitude'],
                                   longitudes=etobicoke_data['Longitude']
                                  )

Islington Avenue
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Westmount
Kingsview Village , St. Phillips , Martin Grove Gardens , Richview Gardens
New Toronto , Mimico South , Humber Bay Shores
South Steeles , Silverstone , Humbergate , Jamestown , Mount Olive , Beaumond Heights , Thistletown , Albion Gardens
Alderwood , Long Branch
Northwest
The Kingsway , Montgomery Road  , Old Mill North
Old Mill South , King's Mill Park , Sunnylea , Humber Bay , Mimico NE , The Queensway East , Royal York South East , Kingsway Park South East
Mimico NW , The Queensway West , South of Bloor , Kingsway Park South West , Royal York South West


In [31]:
print(etobicoke_venues.shape)
etobicoke_venues.head()

(110, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Islington Avenue,43.6662,-79.5282,TD Canada Trust,43.662545,-79.531749,Bank
1,Islington Avenue,43.6662,-79.5282,Shoppers Drug Mart,43.663067,-79.531753,Pharmacy
2,Islington Avenue,43.6662,-79.5282,Humber Valley Park,43.664825,-79.524999,Park
3,Islington Avenue,43.6662,-79.5282,Humber Valley Rink,43.664826,-79.524873,Skating Rink
4,Islington Avenue,43.6662,-79.5282,Thorncrest Drug Store,43.662988,-79.531817,Pharmacy


## Returns number of neighborhoods with venues found by FourSquare

In [32]:
etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood , Long Branch",7,7,7,7,7,7
Islington Avenue,6,6,6,6,6,6
"Kingsview Village , St. Phillips , Martin Grove Gardens , Richview Gardens",4,4,4,4,4,4
"Mimico NW , The Queensway West , South of Bloor , Kingsway Park South West , Royal York South West",15,15,15,15,15,15
Northwest,17,17,17,17,17,17
"Old Mill South , King's Mill Park , Sunnylea , Humber Bay , Mimico NE , The Queensway East , Royal York South East , Kingsway Park South East",2,2,2,2,2,2
"South Steeles , Silverstone , Humbergate , Jamestown , Mount Olive , Beaumond Heights , Thistletown , Albion Gardens",12,12,12,12,12,12
"The Kingsway , Montgomery Road , Old Mill North",17,17,17,17,17,17
"West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale",20,20,20,20,20,20


In [33]:
# finds number of unique categories in the dataframe
print('There are {} uniques categories.'.format(len(etobicoke_venues['Venue Category'].unique())))

There are 53 uniques categories.


## Analyzes the avaialble venues in each neighborhood

In [34]:
# one hot encoding
etobicoke_onehot = pd.get_dummies(etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
etobicoke_onehot['Neighborhood'] = etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [etobicoke_onehot.columns[-1]] + list(etobicoke_onehot.columns[:-1])
etobicoke_onehot = etobicoke_onehot[fixed_columns]

etobicoke_onehot.head()

,Neighborhood,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Bowling Alley,Breakfast Spot,Buffet,...,Salon / Barbershop,Sandwich Place,Skating Rink,Smoke Shop,Sporting Goods Shop,Supermarket,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant,Yoga Studio
0,Islington Avenue,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Islington Avenue,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Islington Avenue,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Islington Avenue,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Islington Avenue,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
etobicoke_onehot.shape

(110, 54)

## Finds the mean of frequency of occurence of each neighborhood

In [36]:
etobicoke_grouped = etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
etobicoke_grouped

,Neighborhood,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Beer Store,Bowling Alley,Breakfast Spot,Buffet,...,Salon / Barbershop,Sandwich Place,Skating Rink,Smoke Shop,Sporting Goods Shop,Supermarket,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant,Yoga Studio
0,"Alderwood , Long Branch",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.142857,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
1,Islington Avenue,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.166667,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
2,"Kingsview Village , St. Phillips , Martin Grov...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
3,"Mimico NW , The Queensway West , South of Bloo...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.066667,0.066667,0.00,0.066667
4,Northwest,0.117647,0.058824,0.058824,0.000000,0.0,0.000000,0.117647,0.000000,0.000000,...,0.058824,0.058824,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
5,"Old Mill South , King's Mill Park , Sunnylea ,...",0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
6,"South Steeles , Silverstone , Humbergate , Jam...",0.000000,0.000000,0.000000,0.000000,0.0,0.083333,0.000000,0.000000,0.000000,...,0.000000,0.083333,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000
7,"The Kingsway , Montgomery Road , Old Mill North",0.000000,0.117647,0.117647,0.058824,0.0,0.000000,0.000000,0.117647,0.000000,...,0.000000,0.000000,0.000000,0.117647,0.00,0.00,0.117647,0.000000,0.00,0.000000
8,"West Deane Park , Princess Gardens , Martin Gr...",0.000000,0.050000,0.050000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.05,0.05,0.050000,0.000000,0.05,0.000000
9,Westmount,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.100000,0.000000,0.000000,0.00,0.10,0.000000,0.000000,0.00,0.000000


In [37]:
etobicoke_grouped.shape

(10, 54)

## Prints out the top 5 venues of each neighborhood by frequency of occurence 

In [38]:
num_top_venues = 5

for hood in etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = etobicoke_grouped[etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood , Long Branch----
               venue  freq
0        Pizza Place  0.14
1     Sandwich Place  0.14
2  Convenience Store  0.14
3        Coffee Shop  0.14
4           Pharmacy  0.14


----Islington Avenue----
           venue  freq
0       Pharmacy  0.33
1  Grocery Store  0.17
2           Bank  0.17
3   Skating Rink  0.17
4           Park  0.17


----Kingsview Village , St. Phillips , Martin Grove Gardens , Richview Gardens----
                  venue  freq
0           Coffee Shop  0.50
1              Bus Line  0.25
2         Grocery Store  0.25
3      Asian Restaurant  0.00
4  Gym / Fitness Center  0.00


----Mimico NW , The Queensway West , South of Bloor , Kingsway Park South West , Royal York South West----
                venue  freq
0       Burrito Place  0.20
1         Yoga Studio  0.07
2        Burger Joint  0.07
3   Fish & Chips Shop  0.07
4  Italian Restaurant  0.07


----Northwest----
              venue  freq
0  Asian Restaurant  0.12
1     Bowling Alley  0.12
2

## Sort venues in descending order

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Creates new dataframe containing the sorted neighborhoods

In [40]:
um_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = etobicoke_grouped['Neighborhood']

for ind in np.arange(etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Alderwood , Long Branch",Gym,Sandwich Place,Convenience Store,Coffee Shop,Pub
1,Islington Avenue,Pharmacy,Park,Bank,Grocery Store,Skating Rink
2,"Kingsview Village , St. Phillips , Martin Grov...",Coffee Shop,Grocery Store,Bus Line,Yoga Studio,Caribbean Restaurant
3,"Mimico NW , The Queensway West , South of Bloo...",Burrito Place,Yoga Studio,Buffet,Fish & Chips Shop,Gym
4,Northwest,Asian Restaurant,Pizza Place,Bowling Alley,Salon / Barbershop,Bus Stop
5,"Old Mill South , King's Mill Park , Sunnylea ,...",Park,Baseball Field,Yoga Studio,Fast Food Restaurant,Farmers Market
6,"South Steeles , Silverstone , Humbergate , Jam...",Grocery Store,Fried Chicken Joint,Pizza Place,Discount Store,Hardware Store
7,"The Kingsway , Montgomery Road , Old Mill North",Bakery,Sushi Restaurant,Bank,Smoke Shop,Breakfast Spot
8,"West Deane Park , Princess Gardens , Martin Gr...",Coffee Shop,Cupcake Shop,Gym,Grocery Store,Diner
9,Westmount,Pizza Place,Discount Store,Flea Market,Supermarket,Sandwich Place


## Installs scikit-learn ML package, gets KMeans clustering, and generates clusters

In [41]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in c:\users\sonle\anaconda3\lib\site-packages (0.22.2.post1)


In [42]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

etobicoke_grouped_clustering =etobicoke_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(etobicoke_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 4, 1, 3, 1, 4, 4, 1])

## Adds 'Cluster Labels' column into 'etobicoke_merged' dataframe

In [43]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

etobicoke_merged = etobicoke_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
etobicoke_merged = etobicoke_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

etobicoke_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M9A,Etobicoke,Islington Avenue,43.666200,-79.528200,0.0,Pharmacy,Park,Bank,Grocery Store,Skating Rink
1,M9B,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr...",43.640741,-79.541902,4.0,Coffee Shop,Cupcake Shop,Gym,Grocery Store,Diner
2,M9C,Etobicoke,"Eringate , Bloordale Gardens , Old Burnhamthor...",43.644110,-79.588907,NaN,NaN,NaN,NaN,NaN,NaN
3,M9P,Etobicoke,Westmount,43.694900,-79.532300,1.0,Pizza Place,Discount Store,Flea Market,Supermarket,Sandwich Place
4,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.695166,-79.550890,2.0,Coffee Shop,Grocery Store,Bus Line,Yoga Studio,Caribbean Restaurant


## Since FourSquares could not find any data on two neighorhoods, drop from dataframe
## Formats the dataframe

In [44]:
# Drops the two unknown neighborhood
etobicoke_merged.drop(index = 2, inplace = True)
etobicoke_merged.drop(index = 5, inplace = True)

# DRops the 'PostalCode' column
etobicoke_merged.drop('PostalCode', axis = 1, inplace = True)

# Resets the index
etobicoke_merged.reset_index(drop=True)


,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Etobicoke,Islington Avenue,43.666200,-79.528200,0.0,Pharmacy,Park,Bank,Grocery Store,Skating Rink
1,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr...",43.640741,-79.541902,4.0,Coffee Shop,Cupcake Shop,Gym,Grocery Store,Diner
2,Etobicoke,Westmount,43.694900,-79.532300,1.0,Pizza Place,Discount Store,Flea Market,Supermarket,Sandwich Place
3,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.695166,-79.550890,2.0,Coffee Shop,Grocery Store,Bus Line,Yoga Studio,Caribbean Restaurant
4,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.743200,-79.587600,1.0,Grocery Store,Fried Chicken Joint,Pizza Place,Discount Store,Hardware Store
5,Etobicoke,"Alderwood , Long Branch",43.602100,-79.540200,1.0,Gym,Sandwich Place,Convenience Store,Coffee Shop,Pub
6,Etobicoke,Northwest,43.715486,-79.564072,1.0,Asian Restaurant,Pizza Place,Bowling Alley,Salon / Barbershop,Bus Stop
7,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.651800,-79.507600,4.0,Bakery,Sushi Restaurant,Bank,Smoke Shop,Breakfast Spot
8,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.632500,-79.493900,3.0,Park,Baseball Field,Yoga Studio,Fast Food Restaurant,Farmers Market
9,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.625600,-79.523100,4.0,Burrito Place,Yoga Studio,Buffet,Fish & Chips Shop,Gym


## Changes the Cluster Labels to intergers

In [45]:
etobicoke_merged['Cluster Labels'] = etobicoke_merged['Cluster Labels'].astype(int)
etobicoke_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Etobicoke,Islington Avenue,43.666200,-79.528200,0,Pharmacy,Park,Bank,Grocery Store,Skating Rink
1,Etobicoke,"West Deane Park , Princess Gardens , Martin Gr...",43.640741,-79.541902,4,Coffee Shop,Cupcake Shop,Gym,Grocery Store,Diner
3,Etobicoke,Westmount,43.694900,-79.532300,1,Pizza Place,Discount Store,Flea Market,Supermarket,Sandwich Place
4,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.695166,-79.550890,2,Coffee Shop,Grocery Store,Bus Line,Yoga Studio,Caribbean Restaurant
6,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.743200,-79.587600,1,Grocery Store,Fried Chicken Joint,Pizza Place,Discount Store,Hardware Store


## Visualizes the clusters

In [46]:
# create map
map_clusters = folium.Map(location=[to_latitude, to_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(etobicoke_merged['Latitude'], etobicoke_merged['Longitude'], etobicoke_merged['Neighborhood'], etobicoke_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyzes the first cluster

In [47]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 0, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Islington Avenue,Pharmacy,Park,Bank,Grocery Store,Skating Rink


## Analyzes the second cluster

In [48]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 1, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Westmount,Pizza Place,Discount Store,Flea Market,Supermarket,Sandwich Place
6,"South Steeles , Silverstone , Humbergate , Jam...",Grocery Store,Fried Chicken Joint,Pizza Place,Discount Store,Hardware Store
7,"Alderwood , Long Branch",Gym,Sandwich Place,Convenience Store,Coffee Shop,Pub
8,Northwest,Asian Restaurant,Pizza Place,Bowling Alley,Salon / Barbershop,Bus Stop


## Analyzes the third cluster

In [49]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 2, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,"Kingsview Village , St. Phillips , Martin Grov...",Coffee Shop,Grocery Store,Bus Line,Yoga Studio,Caribbean Restaurant


## Analyzes the fourth cluster

In [50]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 3, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,"Old Mill South , King's Mill Park , Sunnylea ,...",Park,Baseball Field,Yoga Studio,Fast Food Restaurant,Farmers Market


## Analyzes the fifth cluster

In [51]:
etobicoke_merged.loc[etobicoke_merged['Cluster Labels'] == 4, etobicoke_merged.columns[[1] + list(range(5, etobicoke_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,"West Deane Park , Princess Gardens , Martin Gr...",Coffee Shop,Cupcake Shop,Gym,Grocery Store,Diner
9,"The Kingsway , Montgomery Road , Old Mill North",Bakery,Sushi Restaurant,Bank,Smoke Shop,Breakfast Spot
11,"Mimico NW , The Queensway West , South of Bloo...",Burrito Place,Yoga Studio,Buffet,Fish & Chips Shop,Gym
